# Experimento Sup3

Experimento Sup3. Todos los niveles de intensificación, 50 comunidades, 3 niveles de het. Tasa de mortalidad menor.

In [1]:
import numpy as np
import math, os, random
from scipy.integrate import odeint
from copy import deepcopy

In [2]:
%run modelo.py

## Comunidades

In [3]:
files = []
with open('iteraciones.txt', 'r') as file:
    rows = file.readlines()
    for row in rows:
        files.append([row[:20], int(row[-2])])

In [4]:
all_condiciones_iniciales = [np.loadtxt("./comunidades/" + file[0][:20] + "-c.txt") for file in files]
all_tasas_reproduccion = [np.loadtxt("./comunidades/" + file[0][:20] + "-v.txt") for file in files]
all_matriz_interacciones = [np.loadtxt("./comunidades/" + file[0][:20] + '-m.txt') for file in files]

In [5]:
def modificar_matriz(m):
    for i in range(10):
        m[i][i] *= 15
    return m

def modificar_tasas(tasa):
    tasa[tasa > 0] = tasa[tasa > 0] * 20
    return tasa

def modificar_condiciones0(c0):
    c0 = c0 * 10
    return c0

In [6]:
matriz_interacciones = deepcopy(all_matriz_interacciones)
matriz_interacciones = [modificar_matriz(m) for m in matriz_interacciones]

tasas_reproduccion = deepcopy(all_tasas_reproduccion)
tasas_reproduccion = [modificar_tasas(tasa) for tasa in tasas_reproduccion]

condiciones_iniciales = deepcopy(all_condiciones_iniciales)
condiciones_iniciales = [modificar_condiciones0(c) for c in condiciones_iniciales]

In [7]:
random.seed(17)
comunidades = random.sample(range(201), 50)

In [8]:
comunidades = comunidades[0:50]

## Parámetros iniciales

In [9]:
heterogeneidad = [0, 1, 4]

In [10]:
Dispersion = {"b": 0.3,
              "m": 1,
              "i": 1}

Mortalidad = {"b": 0,
              "m": 0.25,
              "i": 0.80}
t_total = 100

## Experimento

In [11]:
for het in heterogeneidad:
    paisajes = np.load("paisajes_h{}.npy".format(het))
    for c in comunidades:
        res = [None] * len(paisajes)
        for p in range(len(paisajes)):
            res[p] = matriz_agroecologica(paisajes[p].tolist(),
                                          matriz_interacciones[c], 
                                          tasas_reproduccion[c], 
                                          condiciones_iniciales[c],
                                          t_total, 
                                          Dispersion,
                                          Mortalidad, 5)
            res[p] = res[p][-1:]

        res = np.array(res)
        np.save("./datos/expS3/expS3_h{}_c{:03d}".format(het, c), res)